In [29]:
import nbformat

# Read your notebook (assuming version 4 for example purposes)
nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

%run ml_project_normalized.ipynb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Unique primary colors:
 1. black
 2. non-black
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
Unique primary colors:
 1. black
 2. non-black
cleaned color
cleaned breed
dropped columns


/tmp/ipykernel_1018529/1199118304.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_series = pd.to_datetime(df['intake_time'], errors='coerce')


cleaned intake time
cleaned intake condition
cleaned age and sex
Unique primary colors:
 1. black
 2. non-black
cleaned color
cleaned breed
Done running ml_project.ipynb.


In [30]:
from imblearn.ensemble import BalancedRandomForestClassifier  # Import the Balanced Random Forest
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, train_test_split
from sklearn.metrics import make_scorer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt

In [31]:
def train_bal_RF_classifier(X_train, y_train, X_test, cat_cols, target_samples_per_class=3000):
    all_cols = X_train.columns.tolist()
    cat_indices = [all_cols.index(col) for col in cat_cols]

    # Define categorical transformer with one-hot encoding
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-Hot Encode categorical features
    ])

    # Combine transformations into a single preprocessor
    preprocessor = ColumnTransformer(
        transformers=[('cat', categorical_transformer, cat_indices)]
    )

    # Create pipeline with the BalancedRandomForestClassifier
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('brf', BalancedRandomForestClassifier(random_state=42, n_jobs=-1))  # Balanced Random Forest
    ])

    # Define hyperparameter search space
    param_dist = {
        'brf__criterion' : ['entropy', 'gini'],
        'brf__n_estimators': [100, 200, 300],
        'brf__max_depth': [10, 20, None], 
        'brf__min_samples_split': [2, 5, 10],
        'brf__min_samples_leaf': [1, 2, 4],
        'brf__max_features' : [None, 2, 3, 5, 'auto', 'log2']
    }

    # Randomized Search with cross-validation
    search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_dist,
        n_iter=10,
        cv=skf,  # Adjust the number of folds for cross-validation if necessary
        scoring='balanced_accuracy',
        verbose=2,
        random_state=42,
        n_jobs=-1
    )

    print(f"\n[INFO] Starting training with {len(X_train)} samples")
    search.fit(X_train, y_train)
    print("[INFO] Training complete.")

    print("Best parameters:", search.best_params_)
    print("Best CV balanced accuracy:", search.best_score_)

    # Optionally evaluate again with cross_val_score if needed
    try:
        cv_scores = cross_val_score(
            search.best_estimator_,
            X_train,
            y_train,
            cv=skf,  # Number of cross-validation splits
            scoring='balanced_accuracy',
            verbose=3
        )
        print("Generalization accuracy (via cross_val_score):", cv_scores.mean())
    except Exception as e:
        print(f"Cross-validation scoring failed: {e}")

    # Final test prediction
    test_predictions = search.best_estimator_.predict(X_test)

    '''
      # Get the feature importances
    importances = search.best_estimator_.feature_importances_

    # Plotting feature importance
    indices = importances.argsort()[::-1]  # Sort feature importances in descending order
    top_n = 10  # Number of top features to plot
    plt.figure(figsize=(10, 6))
    plt.title("Top 10 Feature Importances")
    plt.bar(range(top_n), importances[indices[:top_n]], align="center")
    plt.xticks(range(top_n), X_train.columns[indices[:top_n]], rotation=45)
    plt.tight_layout()
    plt.show()
    '''
    return search.best_estimator_, test_predictions

In [32]:
df_train = bucket_seasons(df_train)
df_test = bucket_seasons(df_test)

df_train = bucket_days(df_train)
df_test = bucket_days(df_test)

df_train['breed'] = df_train['breed'].astype(str)
df_test['breed'] = df_test['breed'].astype(str)

df_train = df_train.drop(columns=['size', 'intake_hour', 'intake_month'])
df_test = df_test.drop(columns=['size', 'intake_hour', 'intake_month'])

In [33]:
cat_cols = ['intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed','season', 'is_mix', 'time_of_day', 'primary_color'] # experimenting with droping color
num_cols = ['log_age', 'intake_year']    # Replace with your actual numerical columns
# freq_cols = ['primary_color'] # for trees

In [34]:
X = df_train.drop(columns=['outcome_type'])
y = df_train['outcome_type']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)

In [35]:
# Encode the target variable.
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
print('Encoding mapping:', le.classes_)

best_model, test_predictions = train_bal_RF_classifier(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    cat_cols=cat_cols
)


# Apply thresholds to the predicted probabilities
# thresholded_preds = apply_thresholds(y_val_proba, thresholds)

# Convert predictions back to original labels.
classification_report_with_accuracy_score(y_test, test_predictions)
#print(importance)

Encoding mapping: ['Adoption' 'Died' 'Euthanasia' 'Return to Owner' 'Transfer']

[INFO] Starting training with 88924 samples
Fitting 5 folds for each of 10 candidates, totalling 50 fits


/u/nneoma/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/u/nneoma/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/u/nneoma/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/u/nneoma/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check i

[CV] END brf__criterion=gini, brf__max_depth=20, brf__max_features=auto, brf__min_samples_leaf=2, brf__min_samples_split=2, brf__n_estimators=300; total time=   0.1s
[CV] END brf__criterion=gini, brf__max_depth=20, brf__max_features=auto, brf__min_samples_leaf=2, brf__min_samples_split=2, brf__n_estimators=300; total time=   0.1s
[CV] END brf__criterion=gini, brf__max_depth=20, brf__max_features=auto, brf__min_samples_leaf=2, brf__min_samples_split=2, brf__n_estimators=300; total time=   0.1s
[CV] END brf__criterion=gini, brf__max_depth=20, brf__max_features=auto, brf__min_samples_leaf=2, brf__min_samples_split=2, brf__n_estimators=300; total time=   0.1s
[CV] END brf__criterion=gini, brf__max_depth=20, brf__max_features=auto, brf__min_samples_leaf=2, brf__min_samples_split=2, brf__n_estimators=300; total time=   0.1s


/u/nneoma/.local/lib/python3.8/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/u/nneoma/.local/lib/python3.8/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/u/nneoma/.local/lib/python3.8/site-packages/imblearn/ensemble/_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(
/u/nneom

[CV] END brf__criterion=gini, brf__max_depth=None, brf__max_features=2, brf__min_samples_leaf=4, brf__min_samples_split=2, brf__n_estimators=100; total time=   2.9s
[CV] END brf__criterion=gini, brf__max_depth=None, brf__max_features=2, brf__min_samples_leaf=4, brf__min_samples_split=2, brf__n_estimators=100; total time=   2.9s
[CV] END brf__criterion=gini, brf__max_depth=None, brf__max_features=2, brf__min_samples_leaf=4, brf__min_samples_split=2, brf__n_estimators=100; total time=   3.0s
[CV] END brf__criterion=gini, brf__max_depth=None, brf__max_features=2, brf__min_samples_leaf=4, brf__min_samples_split=2, brf__n_estimators=100; total time=   3.1s


...

/u/nneoma/.local/lib/python3.8/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/u/nneoma/.local/lib/python3.8/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/u/nneoma/.local/lib/python3.8/site-packages/imblearn/ensemble/_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(


[CV] END ................................ score: (test=0.456) total time=   2.8s


/u/nneoma/.local/lib/python3.8/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/u/nneoma/.local/lib/python3.8/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/u/nneoma/.local/lib/python3.8/site-packages/imblearn/ensemble/_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(


[CV] END ................................ score: (test=0.457) total time=   2.8s
Generalization accuracy (via cross_val_score): 0.4488648125136563
              precision    recall  f1-score   support

           0       0.66      0.45      0.53     11031
           1       0.03      0.42      0.05       206
           2       0.18      0.52      0.27       697
           3       0.37      0.71      0.49      3236
           4       0.54      0.25      0.34      7061

    accuracy                           0.42     22231
   macro avg       0.35      0.47      0.34     22231
weighted avg       0.56      0.42      0.45     22231



0.4676450510529322